# YASA(複数データの正解率)

Output: acc_yasa_3ch.csv  

Channels:
- EEG(Fpz-Cz)  
- EOG(horizontal)  
- EMG(submental)

In [1]:
import mne
import re
import numpy as np
import pandas as pd
import yasa
from mne.datasets.sleep_physionet.age import fetch_data
from sklearn.metrics import accuracy_score

annotation_desc_2_event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 4,
    "Sleep stage R": 5,
}

# create a new event_id that unifies stages 3 and 4
event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3/4": 4,
    "Sleep stage R": 5,
}


def fetch(amount):
    # [[edf1, edf2, ...], [hypno1, hypno2, ...]]
    DataList = [[], []]

    NAN_data = [39, 68, 69, 78, 79]
    for NAN in NAN_data:
        if amount - 1 > NAN:
            amount += 1

    for i in range(1, amount + 1):
        [data] = fetch_data(subjects=[i], recording=[1])
        edf = mne.io.read_raw_edf(
            data[0], stim_channel="Event marker", misc=["Temp rectal"]
        )

        annot = mne.read_annotations(data[1])

        annot.crop(annot[1]['onset'] - 30 * 60,
                annot[-2]['onset'] + 30 * 60)

        edf.set_annotations(annot, emit_warning=False)

        events, _ = mne.events_from_annotations(
            edf, event_id=annotation_desc_2_event_id, chunk_duration=30.0
        )

        tmax = 30.0 - 1.0 / edf.info["sfreq"]

        epochs = mne.Epochs(
            raw=edf,
            events=events,
            event_id=event_id,
            tmin=0.0,
            tmax=tmax,
            baseline=None,
        )

        hypno = epochs.events[:, 2] - 1

        edf.crop(tmin=float(annot[1]['onset'] - 30 * 60), tmax=float(annot[-2]['onset'] + 40 * 60))
        
        DataList[0].append(edf)
        DataList[1].append(hypno)

    return DataList


amount = 30
Data = fetch(amount)
print(Data)

In [ ]:
accuracy = []

for i in range(amount):
    sls = yasa.SleepStaging(Data[0][i], eeg_name="EEG Fpz-Cz", eog_name="EOG horizontal", emg_name="EMG submental")
    hypno_pred = sls.predict()
    hypno_pred = yasa.hypno_str_to_int(hypno_pred)

    hypno = Data[1][i]
    length = -(len(hypno_pred) - len(hypno))
    try:
        acc = accuracy_score(hypno, hypno_pred[:length])  # 正解率
        print(str(Data[0][i]))
        text = re.search("SC.*-PSG.edf", str(Data[0][i]))
        tmp = [acc, text.group()]
        accuracy.append(tmp)
    except ValueError:
        pass

csv_pre = pd.DataFrame(accuracy)
csv_pre.to_csv("./acc_yasa_3ch.csv")
